<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 5장 연습문제 풀이

In [1]:
from importlib.metadata import version

pkgs = ["numpy", 
        "tiktoken", 
        "torch",
        "tensorflow" # For OpenAI's pretrained weights
       ]
for p in pkgs:
    print(f"{p} version: {version(p)}")

numpy version: 1.26.4
tiktoken version: 0.7.0
torch version: 2.4.0
tensorflow version: 2.16.1


# 연습문제 5.1: 온도 스케일링된 소프트맥스 점수와 샘플링 확률

- 이 섹션에서 정의한 `print_sampled_tokens` 함수를 사용하여 "pizza"라는 단어가 샘플링된 횟수를 출력할 수 있습니다
- 5.3.1절에서 정의한 코드로 시작해봅시다

- 온도가 0이나 0.1일 때는 0번 샘플링되고, 온도가 5로 증가하면 32번 샘플링됩니다. 추정 확률은 32/1000 * 100% = 3.2%입니다

- 실제 확률은 4.3%이며 재조정된 소프트맥스 확률 텐서에 포함되어 있습니다 (`scaled_probas[2][6]`)

- 다음은 5장의 코드를 사용한 독립적인 예제입니다:

In [2]:
import torch

vocab = { 
    "closer": 0,
    "every": 1, 
    "effort": 2, 
    "forward": 3,
    "inches": 4,
    "moves": 5, 
    "pizza": 6,
    "toward": 7,
    "you": 8,
} 
inverse_vocab = {v: k for k, v in vocab.items()}

next_token_logits = torch.tensor(
    [4.51, 0.89, -1.90, 6.75, 1.63, -1.62, -1.89, 6.28, 1.79]
)

def print_sampled_tokens(probas):
    torch.manual_seed(123)
    sample = [torch.multinomial(probas, num_samples=1).item() for i in range(1_000)]
    sampled_ids = torch.bincount(torch.tensor(sample))
    for i, freq in enumerate(sampled_ids):
        print(f"{freq} x {inverse_vocab[i]}")


def softmax_with_temperature(logits, temperature):
    scaled_logits = logits / temperature
    return torch.softmax(scaled_logits, dim=0)


temperatures = [1, 0.1, 5]  # Original, higher, and lower temperature
scaled_probas = [softmax_with_temperature(next_token_logits, T) for T in temperatures]

- 이제 `scaled_probas`를 순회하며 각 경우의 샘플링 빈도를 출력할 수 있습니다:

In [3]:
for i, probas in enumerate(scaled_probas):
    print("\n\nTemperature:", temperatures[i])
    print_sampled_tokens(probas)



Temperature: 1
73 x closer
0 x every
0 x effort
582 x forward
2 x inches
0 x moves
0 x pizza
343 x toward


Temperature: 0.1
0 x closer
0 x every
0 x effort
985 x forward
0 x inches
0 x moves
0 x pizza
15 x toward


Temperature: 5
165 x closer
75 x every
42 x effort
239 x forward
71 x inches
46 x moves
32 x pizza
227 x toward
103 x you


- 샘플링은 "pizza"라는 단어가 샘플링될 때 실제 확률의 근사치를 제공합니다
- 예를 들어, 1000번 중 32번 샘플링되면 추정 확률은 3.2%입니다
- 실제 확률을 얻으려면 `scaled_probas`의 해당 항목에 직접 접근하여 확률을 확인할 수 있습니다

- "pizza"는 어휘에서 7번째 항목이므로, 온도가 5일 때 다음과 같이 얻을 수 있습니다:

In [4]:
temp5_idx = 2
pizza_idx = 6

scaled_probas[temp5_idx][pizza_idx]

tensor(0.0430)

온도가 5로 설정되면 "pizza"라는 단어가 샘플링될 확률은 4.3%입니다

# 연습문제 5.2: 다양한 온도와 top-k 설정

- 온도와 top-k 설정은 개별 LLM에 따라 조정되어야 합니다 (원하는 출력을 생성할 때까지 시행착오 과정이 필요합니다)
- 바람직한 결과는 애플리케이션별로 다릅니다
  - 낮은 top-k와 온도는 덜 무작위적인 결과를 생성하며, 교육 콘텐츠, 기술 문서 작성, 질의응답, 데이터 분석, 코드 생성 등에 적합합니다
  - 높은 top-k와 온도는 더 다양하고 무작위적인 출력을 생성하며, 브레인스토밍 작업, 창의적 글쓰기 등에 더 적합합니다

# 연습문제 5.3: 디코딩 함수의 결정론적 동작

`generate` 함수에서 결정론적 동작을 강제하는 여러 방법이 있습니다:

1. `temperature=0.0`으로 설정
2. `top_k=1`로 설정

다음은 5장의 코드를 사용한 독립적인 예제입니다:

In [ ]:
import tiktoken
import torch
from previous_chapters import GPTModel


GPT_CONFIG_124M = {
    "vocab_size": 50257,  # 어휘 크기
    "context_length": 256,       # 단축된 컨텍스트 길이 (원본: 1024)
    "emb_dim": 768,       # 임베딩 차원
    "n_heads": 12,        # 어텐션 헤드 수
    "n_layers": 12,       # 레이어 수
    "drop_rate": 0.1,     # 드롭아웃 비율
    "qkv_bias": False     # Query-Key-Value 편향
}


torch.manual_seed(123)

tokenizer = tiktoken.get_encoding("gpt2")
model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(torch.load("model.pth", weights_only=True))
model.eval();

In [6]:
from gpt_generate import generate, text_to_token_ids, token_ids_to_text
from previous_chapters import generate_text_simple

In [ ]:
# torch.argmax를 사용하는 결정론적 함수

start_context = "Every effort moves you"

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=25,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("출력 텍스트:\n", token_ids_to_text(token_ids, tokenizer))

In [ ]:
# 결정론적 동작: top_k 없음, 온도 스케일링 없음

token_ids = generate(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=25,
    context_size=GPT_CONFIG_124M["context_length"],
    top_k=None,
    temperature=0.0
)

print("출력 텍스트:\n", token_ids_to_text(token_ids, tokenizer))

- 이전 코드 셀을 다시 실행하면 정확히 동일한 생성된 텍스트가 생성됩니다:

In [ ]:
# 결정론적 동작: top_k 없음, 온도 스케일링 없음

token_ids = generate(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=25,
    context_size=GPT_CONFIG_124M["context_length"],
    top_k=None,
    temperature=0.0
)

print("출력 텍스트:\n", token_ids_to_text(token_ids, tokenizer))

# 연습문제 5.4: 사전학습 계속하기

- 5장에서 모델을 처음 훈련한 Python 세션에 여전히 있다면, 한 에폭 더 사전학습을 계속하려면 메인 장에서 저장한 모델과 옵티마이저를 로드하고 `train_model_simple` 함수를 다시 호출하면 됩니다

- 이 새로운 코드 환경에서 재현 가능하게 만들려면 몇 단계가 더 필요합니다
- 먼저 토크나이저, 모델, 옵티마이저를 로드합니다:

In [ ]:
import tiktoken
import torch
from previous_chapters import GPTModel


GPT_CONFIG_124M = {
    "vocab_size": 50257,   # 어휘 크기
    "context_length": 256, # 단축된 컨텍스트 길이 (원본: 1024)
    "emb_dim": 768,        # 임베딩 차원
    "n_heads": 12,         # 어텐션 헤드 수
    "n_layers": 12,        # 레이어 수
    "drop_rate": 0.1,      # 드롭아웃 비율
    "qkv_bias": False      # Query-Key-Value 편향
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = tiktoken.get_encoding("gpt2")

checkpoint = torch.load("model_and_optimizer.pth", weights_only=True)
model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(checkpoint["model_state_dict"])
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
model.train();

- 다음으로 데이터 로더를 초기화합니다:

In [ ]:
import os
import urllib.request
from previous_chapters import create_dataloader_v1


file_path = "the-verdict.txt"
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"

if not os.path.exists(file_path):
    with urllib.request.urlopen(url) as response:
        text_data = response.read().decode('utf-8')
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text_data)
else:
    with open(file_path, "r", encoding="utf-8") as file:
        text_data = file.read()


# 훈련/검증 비율
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]


torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

- 마지막으로 `train_model_simple` 함수를 사용하여 모델을 훈련합니다:

In [12]:
from gpt_train import train_model_simple

num_epochs = 1
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="Every effort moves you", tokenizer=tokenizer
)

Ep 1 (Step 000000): Train loss 0.271, Val loss 6.545
Ep 1 (Step 000005): Train loss 0.244, Val loss 6.614
Every effort moves you?"  "Yes--quite insensible to the irony. She wanted him vindicated--and by me!"  He laughed again, and threw back his head to look up at the sketch of the donkey. "There were days when I


# 연습문제 5.5: 사전학습된 모델의 훈련 및 검증 세트 손실

- 다음 코드를 사용하여 GPT 모델의 훈련 및 검증 세트 손실을 계산할 수 있습니다:

```python
train_loss = calc_loss_loader(train_loader, gpt, device)
val_loss = calc_loss_loader(val_loader, gpt, device)
```

- 124M 파라미터의 결과 손실은 다음과 같습니다:

```
Training loss: 3.754748503367106
Validation loss: 3.559617757797241
```

- 주요 관찰 사항은 훈련 및 검증 세트 성능이 비슷한 범위에 있다는 것입니다
- 이는 여러 가지로 설명할 수 있습니다:

1. The Verdict는 OpenAI가 GPT-2를 훈련할 때 사전학습 데이터셋의 일부가 아니었습니다. 따라서 모델은 훈련 세트에 명시적으로 과적합되지 않으며 The Verdict의 훈련 및 검증 세트 부분에서 비슷하게 잘 수행됩니다. (검증 세트 손실이 훈련 세트 손실보다 약간 낮은데, 이는 딥러닝에서는 이례적입니다. 그러나 데이터셋이 비교적 작기 때문에 무작위 노이즈 때문일 가능성이 높습니다. 실제로 과적합이 없다면 훈련 및 검증 세트 성능은 대략 동일할 것으로 예상됩니다).

2. The Verdict가 GPT-2의 훈련 데이터셋의 일부였다면, 검증 세트도 훈련에 사용되었을 것이므로 모델이 훈련 데이터에 과적합되었는지 알 수 없습니다. 과적합 정도를 평가하려면 OpenAI가 GPT-2 훈련을 완료한 후 생성된 새로운 데이터셋이 필요합니다. 이렇게 하면 사전학습의 일부가 아니었음을 확인할 수 있습니다.

아래 코드는 이 새 노트북을 위한 재현 가능한 독립 실행형 예제입니다.

In [ ]:
import tiktoken
import torch
from previous_chapters import GPTModel


GPT_CONFIG_124M = {
    "vocab_size": 50257,   # 어휘 크기
    "context_length": 256, # 단축된 컨텍스트 길이 (원본: 1024)
    "emb_dim": 768,        # 임베딩 차원
    "n_heads": 12,         # 어텐션 헤드 수
    "n_layers": 12,        # 레이어 수
    "drop_rate": 0.1,      # 드롭아웃 비율
    "qkv_bias": False      # Query-Key-Value 편향
}


torch.manual_seed(123)

tokenizer = tiktoken.get_encoding("gpt2")

In [14]:
from gpt_download import download_and_load_gpt2

settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")

File already exists and is up-to-date: gpt2/124M/checkpoint
File already exists and is up-to-date: gpt2/124M/encoder.json
File already exists and is up-to-date: gpt2/124M/hparams.json
File already exists and is up-to-date: gpt2/124M/model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2/124M/model.ckpt.index
File already exists and is up-to-date: gpt2/124M/model.ckpt.meta
File already exists and is up-to-date: gpt2/124M/vocab.bpe


In [ ]:
# 간결성을 위해 딕셔너리에 모델 구성 정의
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

# 기본 구성을 복사하고 특정 모델 설정으로 업데이트
model_name = "gpt2-small (124M)"  # 예제 모델 이름
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])
NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})

gpt = GPTModel(NEW_CONFIG)
gpt.eval();

In [16]:
from gpt_generate import load_weights_into_gpt


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
load_weights_into_gpt(gpt, params)
gpt.to(device);

In [ ]:
import os
import urllib.request
from previous_chapters import create_dataloader_v1


file_path = "the-verdict.txt"
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"

if not os.path.exists(file_path):
    with urllib.request.urlopen(url) as response:
        text_data = response.read().decode('utf-8')
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text_data)
else:
    with open(file_path, "r", encoding="utf-8") as file:
        text_data = file.read()


# 훈련/검증 비율
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]


torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [18]:
from gpt_train import calc_loss_loader

torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader
train_loss = calc_loss_loader(train_loader, gpt, device)
val_loss = calc_loss_loader(val_loader, gpt, device)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

Training loss: 3.7547486888037787
Validation loss: 3.5596182346343994


가장 큰 GPT-2 모델에 대해서도 이를 반복할 수 있지만, 컨텍스트 길이를 업데이트하는 것을 잊지 마세요:

In [19]:
settings, params = download_and_load_gpt2(model_size="1558M", models_dir="gpt2")

model_name = "gpt2-xl (1558M)"
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])
NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})

gpt = GPTModel(NEW_CONFIG)
gpt.eval()

load_weights_into_gpt(gpt, params)
gpt.to(device)

torch.manual_seed(123)
train_loss = calc_loss_loader(train_loader, gpt, device)
val_loss = calc_loss_loader(val_loader, gpt, device)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

checkpoint: 100%|███████████████████████████| 77.0/77.0 [00:00<00:00, 43.5kiB/s]
encoder.json: 100%|███████████████████████| 1.04M/1.04M [00:00<00:00, 2.75MiB/s]
hparams.json: 100%|█████████████████████████| 91.0/91.0 [00:00<00:00, 60.2kiB/s]
model.ckpt.data-00000-of-00001: 100%|█████| 6.23G/6.23G [06:02<00:00, 17.2MiB/s]
model.ckpt.index: 100%|████████████████████| 20.7k/20.7k [00:00<00:00, 171kiB/s]
model.ckpt.meta: 100%|████████████████████| 1.84M/1.84M [00:00<00:00, 4.27MiB/s]
vocab.bpe: 100%|████████████████████████████| 456k/456k [00:00<00:00, 1.73MiB/s]


Training loss: 3.3046312861972384
Validation loss: 3.1195147037506104


# 연습문제 5.6: 더 큰 모델 시도하기

- 메인 장에서는 124M 파라미터만 있는 가장 작은 GPT-2 모델을 실험했습니다
- 그 이유는 리소스 요구사항을 최대한 낮게 유지하기 위해서였습니다
- 그러나 최소한의 코드 변경으로 더 큰 모델을 쉽게 실험할 수 있습니다
- 예를 들어, 5장에서 124M 대신 1558M 모델을 로드하려면 변경해야 할 코드는 다음 2줄뿐입니다

```python
settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")
model_name = "gpt2-small (124M)"
```

- 업데이트된 코드는 다음과 같습니다


```python
settings, params = download_and_load_gpt2(model_size="1558M", models_dir="gpt2")
model_name = "gpt2-xl (1558M)"
```

In [ ]:
import tiktoken
import torch
from previous_chapters import GPTModel


GPT_CONFIG_124M = {
    "vocab_size": 50257,   # 어휘 크기
    "context_length": 256, # 단축된 컨텍스트 길이 (원본: 1024)
    "emb_dim": 768,        # 임베딩 차원
    "n_heads": 12,         # 어텐션 헤드 수
    "n_layers": 12,        # 레이어 수
    "drop_rate": 0.1,      # 드롭아웃 비율
    "qkv_bias": False      # Query-Key-Value 편향
}


tokenizer = tiktoken.get_encoding("gpt2")

In [21]:
from gpt_download import download_and_load_gpt2
from gpt_generate import load_weights_into_gpt


model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

model_name = "gpt2-xl (1558M)"
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])
NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})

gpt = GPTModel(NEW_CONFIG)
gpt.eval()

settings, params = download_and_load_gpt2(model_size="1558M", models_dir="gpt2")
load_weights_into_gpt(gpt, params)

File already exists and is up-to-date: gpt2/1558M/checkpoint
File already exists and is up-to-date: gpt2/1558M/encoder.json
File already exists and is up-to-date: gpt2/1558M/hparams.json
File already exists and is up-to-date: gpt2/1558M/model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2/1558M/model.ckpt.index
File already exists and is up-to-date: gpt2/1558M/model.ckpt.meta
File already exists and is up-to-date: gpt2/1558M/vocab.bpe


In [22]:
from gpt_generate import generate, text_to_token_ids, token_ids_to_text

In [23]:
torch.manual_seed(123)

token_ids = generate(
    model=gpt,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=25,
    context_size=NEW_CONFIG["context_length"],
    top_k=50,
    temperature=1.5
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you toward finding an ideal life. You don't have to accept your current one at once, because if you do you'll never
